# Import libraries

In [ ]:
import torch
import os

from sympy.matrices.expressions.tests.test_slice import test_entry
from sympy.utilities.tests.test_lambdify import test_integral
from torch.utils.data import DataLoader
from segmentation_models_pytorch.utils.metrics import IoU
from tqdm import tqdm

from src.utils import save_to_hdf5, load_hdf5, load_model, multiclass_iou
from src.datasets import ExcavatorDataset
from src.config import IMAGE_SIZE, TRANSFORMER, DEVICE, ROOT
from src.evaluate import compute_and_save_confidence_vectors
from models.Segmentation import DeepLabV3Model, DeepLabV3PlusModel, PyramidAttentionNetworkModel, UNetModel

# Root

In [ ]:
root = ROOT
batch_size = 1

# Initialize models

**Note**: UNEt performs quite badly (only achieves `val IoU` of 0.78).

In [ ]:
# DeepLabV3
dlv3 =DeepLabV3Model(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3_HybridFocalDiceLoss.pt')

# DeepLabV3Plus
dlv3p = DeepLabV3PlusModel(model_path=f'{ROOT}/models/torch_model_files/DeepLabV3Plus_HybridFocalDiceLoss.pt')

# UNet
unet = UNetModel(model_path=f'{ROOT}/models/torch_model_files/UNet_HybridFocalDiceLoss.pt')

# Pyramid Attention Network
pan = PyramidAttentionNetworkModel(model_path=f'{ROOT}/models/torch_model_files/PyramidAttentionNetwork_HybridFocalDiceLoss.pt')

# Pass a model to its corresponding ioU file
iou_paths = {
    dlv3: {
        'train': f'{ROOT}/res/model_performance/train_iou_dlv3.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_dlv3.h5'
    },
    dlv3p: {
        'train': f'{ROOT}/res/model_performance/train_iou_dlv3p.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_dlv3p.h5'
    },
    unet: {
        'train': f'{ROOT}/res/model_performance/train_iou_unet.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_unet.h5'
    },
    pan: {
        'train': f'{ROOT}/res/model_performance/train_iou_pan.h5',
        'val': f'{ROOT}/res/model_performance/val_iou_pan.h5'
    }
}

# Confidence paths
confidence_paths = {
    dlv3: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3_no_bg_val.h5'
        },
        'only_classes_in_train': f'{ROOT}/res/confidence_vectors/dlv3_only_classes_in_train.h5'
    },
    dlv3p: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3p_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3p_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/dlv3p_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/dlv3p_no_bg_val.h5'
        },
        'only_classes_in_train': f'{ROOT}/res/confidence_vectors/dlv3p_only_classes_in_train.h5'
    },
    unet: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/unet_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/unet_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/unet_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/unet_no_bg_val.h5'
        },
        'only_classes_in_train': f'{ROOT}/res/confidence_vectors/unet_only_classes_in_train.h5'
    },
    pan: {
        'with_background': {
            'train': f'{ROOT}/res/confidence_vectors/pan_with_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/pan_with_bg_val.h5'
        },
        'no_background': {
            'train': f'{ROOT}/res/confidence_vectors/pan_no_bg_train.h5',
            'val': f'{ROOT}/res/confidence_vectors/pan_no_bg_val.h5'
        },
        'only_classes_in_train': f'{ROOT}/res/confidence_vectors/pan_only_classes_in_train.h5'
    }
}

# Load dataset

In [ ]:
train_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='train', transform=TRANSFORMER)
print("Number of training samples:", num_train_imgs:=len(train_dataset))
test_dataset = ExcavatorDataset(return_type='image+mask+path', purpose='test', transform=TRANSFORMER)
print("Number of test samples:", num_test_imgs:=len(test_dataset))

# Compute and save predicted masks

In [ ]:
train_iou_dlv3 = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_iou_dlv3p = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_iou_unet = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_iou_pan = torch.zeros(num_train_imgs, dtype=torch.float32, device=DEVICE)
train_paths = []

test_iou_dlv3 = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_iou_dlv3p = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_iou_unet = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_iou_pan = torch.zeros(num_test_imgs, dtype=torch.float32, device=DEVICE)
test_paths = []

In [ ]:

# Compute predicted masks for training set
for i, (imgs, masks, paths) in tqdm(enumerate(train_dataset)):
    imgs = imgs.to(DEVICE).unsqueeze(0)
    masks = masks.to(DEVICE)
    output_dlv3 = dlv3(imgs).squeeze(0)
    output_dlv3p = dlv3p(imgs).squeeze(0)
    output_unet = unet(imgs).squeeze(0)
    output_pan = pan(imgs).squeeze(0)
    train_iou_dlv3[i] = multiclass_iou(output_dlv3, masks)
    train_iou_dlv3p[i] = multiclass_iou(output_dlv3p, masks)
    train_iou_unet[i] = multiclass_iou(output_unet, masks)
    train_iou_pan[i] = multiclass_iou(output_pan, masks)
    train_paths.append(paths)

train_paths = [path.replace('|', '/') for path in train_paths]
save_to_hdf5(f'{root}/res/model_performance/train_iou_dlv3.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_dlv3)})
save_to_hdf5(f'{root}/res/model_performance/train_iou_dlv3p.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_dlv3p)})
save_to_hdf5(f'{root}/res/model_performance/train_iou_unet.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_unet)})
save_to_hdf5(f'{root}/res/model_performance/train_iou_pan.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(train_paths, train_iou_pan)})

# Compute predicted masks for validation set
for i, (imgs, masks, paths) in tqdm(enumerate(test_dataset)):
    imgs = imgs.to(DEVICE).unsqueeze(0)
    masks = masks.to(DEVICE)
    output_dlv3 = dlv3(imgs).squeeze(0)
    output_dlv3p = dlv3p(imgs).squeeze(0)
    output_unet = unet(imgs).squeeze(0)
    output_pan = pan(imgs).squeeze(0)
    test_iou_dlv3[i] = multiclass_iou(output_dlv3, masks)
    test_iou_dlv3p[i] = multiclass_iou(output_dlv3p, masks)
    test_iou_unet[i] = multiclass_iou(output_unet, masks)
    test_iou_pan[i] = multiclass_iou(output_pan, masks)
    test_paths.append(paths)

test_paths = [path.replace('|', '/') for path in test_paths]
save_to_hdf5(f'{root}/res/model_performance/val_iou_dlv3.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_dlv3)})
save_to_hdf5(f'{root}/res/model_performance/val_iou_dlv3p.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_dlv3p)})
save_to_hdf5(f'{root}/res/model_performance/val_iou_unet.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_unet)})
save_to_hdf5(f'{root}/res/model_performance/val_iou_pan.h5', {os.path.basename(pths): iou.cpu().numpy() for pths, iou in zip(test_paths, test_iou_pan)})


# Compute and save Prediction Confidence for all models

### A) All classes are considered, background included

In [ ]:
compute_and_save_confidence_vectors(dlv3,
                                    train_dataset,
                                    output_dir=confidence_paths[dlv3]['with_background']['train'])
compute_and_save_confidence_vectors(dlv3p,
                                    train_dataset,
                                    output_dir=confidence_paths[dlv3p]['with_background']['train'])
compute_and_save_confidence_vectors(unet,
                                    train_dataset,
                                    output_dir=confidence_paths[unet]['with_background']['train'])
compute_and_save_confidence_vectors(pan,
                                    train_dataset,
                                    output_dir=confidence_paths[pan]['with_background']['train'])

compute_and_save_confidence_vectors(dlv3,
                                    test_dataset,
                                    output_dir=confidence_paths[dlv3]['with_background']['val'])
compute_and_save_confidence_vectors(dlv3p,
                                    test_dataset,
                                    output_dir=confidence_paths[dlv3p]['with_background']['val'])
compute_and_save_confidence_vectors(unet,
                                    test_dataset,
                                    output_dir=confidence_paths[unet]['with_background']['val'])
compute_and_save_confidence_vectors(pan,
                                    test_dataset,
                                    output_dir=confidence_paths[pan]['with_background']['val'])

## B) All classes are considered, background excluded

In [ ]:
compute_and_save_confidence_vectors(dlv3,
                                    train_dataset,
                                    ignore_background=True,
                                    output_dir=confidence_paths[dlv3]['no_background']['train'])
compute_and_save_confidence_vectors(dlv3p,
                                    train_dataset,
                                    ignore_background=True,
                                    output_dir=confidence_paths[dlv3p]['no_background']['train'])
compute_and_save_confidence_vectors(unet,
                                    train_dataset,
                                    ignore_background=True,
                                    output_dir=confidence_paths[unet]['no_background']['train'])
compute_and_save_confidence_vectors(pan,
                                    train_dataset,
                                    output_dir=confidence_paths[pan]['no_background']['train'])

compute_and_save_confidence_vectors(dlv3,
                                    test_dataset,
                                    ignore_background=True,
                                    output_dir=confidence_paths[dlv3]['no_background']['val'])
compute_and_save_confidence_vectors(dlv3p,
                                    test_dataset,
                                    ignore_background=True,
                                    output_dir=confidence_paths[dlv3p]['no_background']['val'])
compute_and_save_confidence_vectors(unet,
                                    test_dataset,
                                    output_dir=confidence_paths[unet]['no_background']['val'])
compute_and_save_confidence_vectors(pan,
                                    test_dataset,
                                    ignore_background=True,
                                    output_dir=confidence_paths[pan]['no_background']['val'])

## C) Only classes in the training masks are considered

In [ ]:
compute_and_save_confidence_vectors(dlv3,
                                    train_dataset,
                                    only_classes_in_gt_mask=True,
                                    output_dir=confidence_paths[dlv3]['only_classes_in_train'])
compute_and_save_confidence_vectors(dlv3p,
                                    train_dataset,
                                    only_classes_in_gt_mask=True,
                                    output_dir=confidence_paths[dlv3p]['only_classes_in_train'])
compute_and_save_confidence_vectors(unet,
                                    train_dataset,
                                    only_classes_in_gt_mask=True,
                                    output_dir=confidence_paths[unet]['only_classes_in_train'])
compute_and_save_confidence_vectors(pan,
                                    train_dataset,
                                    only_classes_in_gt_mask=True,
                                    output_dir=confidence_paths[pan]['only_classes_in_train'])